<a href="https://colab.research.google.com/github/flaghunter21/DFS/blob/main/MLB_Tiers.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Finding all Combinations of MLB Tiers Lineups

In [101]:
import pandas as pd
import itertools
from itertools import combinations

In [102]:
df = pd.read_csv('tiers.csv')
df.head()

,Player,Inj,Likes,Pos,Tier,Order,Hand,Team,Opp,Time,...,K/9,WHIP,Avg,STDV,Ceiling,FC Proj,My Proj,Exp.,Used,Con.
0,Austin Riley,NaN,NaN,3B,T3,3,R,ATL,vs SF,6:20 PM,...,0,0,8.7,8.58,31.12,13.96,13.96,100,NaN,51
1,Dansby Swanson,NaN,NaN,SS,T3,2,R,ATL,vs SF,6:20 PM,...,0,0,8.6,7.43,28.66,13.80,13.80,100,NaN,57
2,Ronald Acuna,NaN,NaN,OF,T1,1,R,ATL,vs SF,6:20 PM,...,0,0,9.5,7.50,29.71,14.71,14.71,100,NaN,61
3,Cedric Mullins,NaN,NaN,OF,T6,1,L,BAL,vs WSH,6:05 PM,...,0,0,7.7,7.06,23.18,9.06,9.06,100,NaN,54
4,Rafael Devers,NaN,NaN,3B,T2,2,L,BOS,vs DET,6:10 PM,...,0,0,10.2,7.72,30.10,14.66,14.66,100,NaN,62


In [114]:
print(df.columns)
df = df.loc[:,['Player', 'Tier','My Proj']]
df.sort_values('Tier').head()

Index(['Player', 'Tier', 'My Proj'], dtype='object')


,Player,Tier,My Proj
34,Vladimir Guerrero Jr.,T1,12.39
2,Ronald Acuna,T1,14.71
20,Aaron Judge,T1,12.84
10,Jose Altuve,T1,12.28
4,Rafael Devers,T2,14.66


In [104]:
tier_1 = df[df['Tier'] == 'T1']['Player']
tier_2 = df[df['Tier'] == 'T2']['Player']
tier_3 = df[df['Tier'] == 'T3']['Player']
tier_4 = df[df['Tier'] == 'T4']['Player']
tier_5 = df[df['Tier'] == 'T5']['Player']
tier_6 = df[df['Tier'] == 'T6']['Player']

We now find all possible combinations using `itertools.product`.

In [105]:
combos = list(itertools.product(tier_1, tier_2, tier_3, tier_4, tier_5, tier_6))
print(len(combos))
combos_df = pd.DataFrame(combos, columns = ['T1', 'T2', 'T3', 'T4', 'T5', 'T6'])
combos_df.head()

53760


,T1,T2,T3,T4,T5,T6
0,Ronald Acuna,Rafael Devers,Austin Riley,J.D. Martinez,Trevor Story,Cedric Mullins
1,Ronald Acuna,Rafael Devers,Austin Riley,J.D. Martinez,Trevor Story,Xander Bogaerts
2,Ronald Acuna,Rafael Devers,Austin Riley,J.D. Martinez,Trevor Story,Willson Contreras
3,Ronald Acuna,Rafael Devers,Austin Riley,J.D. Martinez,Trevor Story,Michael Brantley
4,Ronald Acuna,Rafael Devers,Austin Riley,J.D. Martinez,Trevor Story,Christian Yelich


There are 53,760 possible lineup combinations for this slate. We can map the projections to each player to get every lineup's projection.

In [108]:
projection_map = dict(zip(df['Player'], df['My Proj']))
projection_map

{'Aaron Judge': 12.84,
 'Alex Bregman': 11.05,
 'Anthony Rizzo': 11.61,
 'Austin Riley': 13.96,
 'Bo Bichette': 11.19,
 'Byron Buxton': 14.73,
 'Carlos Correa': 12.42,
 'Cedric Mullins': 9.06,
 'Christian Yelich': 9.23,
 'Corey Seager': 10.48,
 'Dansby Swanson': 13.8,
 'George Springer': 11.06,
 'Giancarlo Stanton': 10.64,
 'J.D. Martinez': 13.0,
 'J.T. Realmuto': 8.45,
 'Ji-Man Choi': 0.88,
 'Jose Altuve': 12.28,
 'Juan Soto': 11.49,
 'Kyle Schwarber': 11.11,
 'Kyle Tucker': 11.05,
 'Luis Arraez': 11.96,
 'Luis Robert': 10.08,
 'Marcus Semien': 9.72,
 'Michael Brantley': 10.37,
 'Nolan Arenado': 12.52,
 'Paul Goldschmidt': 14.14,
 'Peter Alonso': 11.24,
 'Rafael Devers': 14.66,
 'Randy Arozarena': 9.4,
 'Rhys Hoskins': 10.87,
 'Ronald Acuna': 14.71,
 'Tommy Edman': 12.82,
 'Trevor Story': 10.27,
 'Vladimir Guerrero Jr.': 12.39,
 'Willson Contreras': 9.27,
 'Willy Adames': 10.17,
 'Xander Bogaerts': 12.39,
 'Yandy Diaz': 8.38}

In [107]:
proj_cols = []
for i in range(1,7):
  combos_df[f'T{i}_Proj'] = combos_df[f'T{i}'].map(projection_map)
  proj_cols.append(f'T{i}_Proj')
combos_df['Total Proj'] = combos_df[proj_cols].sum(axis=1)
combos_df.head()

,T1,T2,T3,T4,T5,T6,T1_Proj,T2_Proj,T3_Proj,T4_Proj,T5_Proj,T6_Proj,Total Proj
0,Ronald Acuna,Rafael Devers,Austin Riley,J.D. Martinez,Trevor Story,Cedric Mullins,14.71,14.66,13.96,13.0,10.27,9.06,75.66
1,Ronald Acuna,Rafael Devers,Austin Riley,J.D. Martinez,Trevor Story,Xander Bogaerts,14.71,14.66,13.96,13.0,10.27,12.39,78.99
2,Ronald Acuna,Rafael Devers,Austin Riley,J.D. Martinez,Trevor Story,Willson Contreras,14.71,14.66,13.96,13.0,10.27,9.27,75.87
3,Ronald Acuna,Rafael Devers,Austin Riley,J.D. Martinez,Trevor Story,Michael Brantley,14.71,14.66,13.96,13.0,10.27,10.37,76.97
4,Ronald Acuna,Rafael Devers,Austin Riley,J.D. Martinez,Trevor Story,Christian Yelich,14.71,14.66,13.96,13.0,10.27,9.23,75.83


In [113]:
combos_df['Total Proj'].describe()

count    53760.000000
mean        69.007762
std          4.498860
min         52.230000
25%         66.560000
50%         69.510000
75%         72.090000
max         81.610000
Name: Total Proj, dtype: float64

The lineup combinations have an average of 69 points. The minimum projection is 52.23 and the maximum projection is 81.61.

In [110]:
combos_df.to_csv('combos.csv', index=False)